# <center>Определение токсичности комментариев</center>

## <center>Введение</center>

Целью работы является создание модели, которая будет находить токсичные комментарии и отправлять их на модерацию. F1-мера должны быть выше 0.75.

## <center>План проекта</center>

**Импорт библиотек**

**Предобработка данных**
+ Чистка с помощью регулярных выражений

**"Тупая" модель**

**Модели**
+ Логистическая регрессия
+ Дерево решений

**Заключение**

## <center>Импорт библиотек</center>

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score, precision_score, f1_score, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

## <center>Предобработка данных</center>

Считаем данные и посмотрим на них:

In [2]:
data = pd.read_csv('toxic_comments.csv')

display(data.head(20))
print(data.info())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


Да, 6-ая строка заставляет пожалеть о том, что знаешь английский :) Но проблема не в токсичности - проблема в мусоре: тегах, ссылках и служебных словах типа `REDIRECT`.

Оценим дисбаланс:

In [3]:
print(data['toxic'].value_counts())

0    143346
1     16225
Name: toxic, dtype: int64


Дисбаланс есть и сильный - но не катастрофически, токсичных комментариев у нас больше 10%. Апсемплить здесь, на мой взгляд, необходимости нет, объектов класса достаточно - будем бороться с дисбалансом другими методами.

### <center>Чистка с помощью регулярных выражений</center>

Переведём тексты в юникод и посмотрим на них повнимательнее:

In [4]:
corpus = data['text'].values.astype('U')

print(len(corpus))
print(corpus[0:20])

159571
["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 '"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style 

Основной мусор - это `\n` и айпишиники, которые оказались здесь неясно как. Нас интересуют только слова, поэтому нужно оставить:
+ Буквы a-z, A-Z;
+ Пробелы;
+ Символ `'`

Символ `'` в английском используется часто в множестве сокращений: `I'm`, `I'd`, `doesn't` и т. д.. И нам не нужно превращать `doesn't` в два слова `doesn` и `t`, вот эти торчащие буквы просто не будут иметь смысла. Поэтому:

In [5]:
cleared_corpus = []

for i in range(len(corpus)):
    temp = re.sub(r"[^a-zA-Z' ]", ' ', corpus[i])
    cleared_corpus.append(' '.join(temp.split()))
    
print(len(cleared_corpus))
for i in range(0, 20):
    print(cleared_corpus[i])
    print()

159571
Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren't vandalisms just closure on some GAs after I voted at New York Dolls FAC And please don't remove the template from the talk page since I'm retired now

D'aww He matches this background colour I'm seemingly stuck with Thanks talk January UTC

Hey man I'm really not trying to edit war It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page He seems to care more about the formatting than the actual info

More I can't make any real suggestions on improvement I wondered if the section statistics should be later on or a subsection of types of accidents I think the references may need tidying so that they are all in the exact same format ie date format etc I can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know There appear

Вот так вроде хорошо - мне кажется, что осталось сделать одну вещь: перевести всё в нижний регистр. Я полагаю, что sklearn достаточно умён, чтобы сделать это своими методами и не считать `cocksucker` и `COCKSUCKER` двумя разными словами, но я не уверена - так что поможем ему:

In [6]:
for i in range(len(cleared_corpus)):
    cleared_corpus[i] = cleared_corpus[i].lower()
    
print(len(cleared_corpus))
for i in range(0, 5):
    print(cleared_corpus[i])
    print()

159571
explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now

d'aww he matches this background colour i'm seemingly stuck with thanks talk january utc

hey man i'm really not trying to edit war it's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info

more i can't make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appear

Отлично - убрать отсюда местоимения, междометия, предлоги и прочие вещи, которые не нужны, по идее должен `stop_words`. Заменим исходные данные предобработанными:

In [7]:
data['text'] = pd.Series(cleared_corpus)

display(data.head(20))
print(data.info())

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww he matches this background colour i'm se...,0
2,hey man i'm really not trying to edit war it's...,0
3,more i can't make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0
5,congratulations from me as well use the tools ...,0
6,cocksucker before you piss around on my work,1
7,your vandalism to the matt shirvington article...,0
8,sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


Получилось - едем дальше :)

## <center>"Тупая" модель</center>

Непонятно, 0.75 - это много или мало; посмотрим на F1-меру в сценарии "все токсичные, всех в бан, полнота 1". Считать здесь TF-IDF - только ресурсы тратить, обойдемся без предобработки:

In [8]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
features_train = train['text']
target_train = train['toxic']
features_test = test['text']
target_test = test['toxic']

model_dummy = DummyClassifier(strategy='constant', constant=1, random_state=42)
model_dummy.fit(features_train, target_train)
predictions = model_dummy.predict(features_test)

recall = recall_score(target_test, predictions)
precision = precision_score(target_test, predictions)
f1_sc = f1_score(target_test, predictions)
print(f'Полнота - {recall: .4f}, точность - {precision: .4f}, F1-мера - {f1_sc: .4f}')

Полнота -  1.0000, точность -  0.1016, F1-мера -  0.1845


Ага - много. Что ж, поехали :)

## <center>Модели</center>

### <center>Логистическая регрессия</center>

Начнём с логистической регрессии - F1 мера должна быть не меньше 0.75 и также хотя бы из интереса мы посмотрим на полноту и точность.

In [9]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

train, test = train_test_split(data, test_size=0.2, random_state=42)
features_train = count_tf_idf.fit_transform(train['text'])
target_train = train['toxic']
features_test = count_tf_idf.transform(test['text'])
target_test = test['toxic']

model_lr = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42) 

In [10]:
%%time
model_lr.fit(features_train, target_train)

Wall time: 2.43 s


LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')

In [11]:
%%time
predictions = model_lr.predict(features_test)

Wall time: 25.9 ms


In [12]:
recall = recall_score(target_test, predictions)
precision = precision_score(target_test, predictions)
f1_sc = f1_score(target_test, predictions)
print(f'Полнота - {recall: .4f}, точность - {precision: .4f}, F1-мера - {f1_sc: .4f}')

Полнота -  0.8533, точность -  0.6740, F1-мера -  0.7531


В целом мы уже получили нужный результат - попробуем добиться лучшего, меняя регуляризацию:

In [13]:
features_train = train['text']
target_train = train['toxic']
features_test = test['text']
target_test = test['toxic']

lr_pipeline = Pipeline(steps=[('prep', count_tf_idf), 
                                ('est', LogisticRegression(solver='liblinear', 
                                                           class_weight='balanced', random_state=42))])
lr_param = {'est__C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

#gs_lr = GridSearchCV(lr_pipeline,
#                      param_grid=lr_param,
#                      scoring=make_scorer(f1_score),
#                      cv=3, verbose=0)
#gs_lr.fit(features_train, target_train)

#print(gs_lr.best_score_)
#print(gs_lr.best_params_)

Лучший результат: `0.7427747396662953
{'est__C': 0.9}`. Сильнее не надо, надо слабее:

In [14]:
lr_param = {'est__C': [1.2, 1.5, 1.7, 2, 2.2, 2.5, 2.7, 3, 3.2]}

#gs_lr = GridSearchCV(lr_pipeline,
#                      param_grid=lr_param,
#                      scoring=make_scorer(f1_score),
#                      cv=3, verbose=0)
#gs_lr.fit(features_train, target_train)

#print(gs_lr.best_score_)
#print(gs_lr.best_params_)

Лучший результат: `0.7594573301134838
{'est__C': 3.2}`. Ещё слабее:

In [15]:
lr_param = {'est__C': [4, 5, 6, 7, 8, 9, 10]}

#gs_lr = GridSearchCV(lr_pipeline,
#                      param_grid=lr_param,
#                      scoring=make_scorer(f1_score),
#                      cv=3, verbose=0)
#gs_lr.fit(features_train, target_train)

#print(gs_lr.best_score_)
#print(gs_lr.best_params_)

Лучший результат: `0.7636692421225026
{'est__C': 10}`. Пробуем дальше:

In [16]:
lr_param = {'est__C': [15, 20, 25, 30, 35, 40, 45, 50]}

#gs_lr = GridSearchCV(lr_pipeline,
#                      param_grid=lr_param,
#                      scoring=make_scorer(f1_score),
#                      cv=3, verbose=0)
#gs_lr.fit(features_train, target_train)

#print(gs_lr.best_score_)
#print(gs_lr.best_params_)

Лучший результат: `0.7627577174824255
{'est__C': 15}`. Хуже, чем при 10 - значит смотрим отрезок от 10 до 15:
</div>

In [17]:
lr_param = {'est__C': [11, 12, 13, 14]}

#gs_lr = GridSearchCV(lr_pipeline,
#                      param_grid=lr_param,
#                      scoring=make_scorer(f1_score),
#                      cv=3, verbose=0)
#gs_lr.fit(features_train, target_train)

#print(gs_lr.best_score_)
#print(gs_lr.best_params_)

Лучший результат: `0.7634608170294283
{'est__C': 12}`. Итого лучшее - 10:

In [18]:
features_train = count_tf_idf.fit_transform(train['text'])
target_train = train['toxic']
features_test = count_tf_idf.transform(test['text'])
target_test = test['toxic']

model_lr = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42, C=10) 

In [19]:
%%time
model_lr.fit(features_train, target_train)

Wall time: 2.5 s


LogisticRegression(C=10, class_weight='balanced', random_state=42,
                   solver='liblinear')

In [20]:
%%time
predictions = model_lr.predict(features_test)

Wall time: 4.99 ms


In [21]:
recall = recall_score(target_test, predictions)
precision = precision_score(target_test, predictions)
f1_sc = f1_score(target_test, predictions)
print(f'Полнота - {recall: .4f}, точность - {precision: .4f}, F1-мера - {f1_sc: .4f}')

Полнота -  0.8280, точность -  0.7299, F1-мера -  0.7759


Результат очень хороший - посмотрим, как справятся другие модели.

### <center>Дерево решений</center>

Здесь уже не обойтись без пайплайна и гридсёрча - отделим тестовую выборку и будем искать лучшее дерево с 5-фолдовой кроссвалидацией:
(Здесь и далее сам поиск в коде закомменчен, потому что занимает много времени)

In [22]:
features_train = train['text']
target_train = train['toxic']
features_test = test['text']
target_test = test['toxic']

tree_pipeline = Pipeline(steps=[('prep', count_tf_idf), 
                                ('est', DecisionTreeClassifier(random_state=42, class_weight='balanced'))])
tree_param = {'est__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10]}

#gs_tree = GridSearchCV(tree_pipeline,
#                      param_grid=tree_param,
#                      scoring=make_scorer(f1_score),
#                      cv=5, verbose=0)
#gs_tree.fit(features_train, target_train)

#print(gs_tree.best_score_)
#print(gs_tree.best_params_)

Лучший результат: `0.5454164639462677
{'est__max_depth': 10}`... Что вообще-то фантастически плохо - здесь хочется выжать тест и посмотреть на полноту с точностью:

In [23]:
features_train = count_tf_idf.fit_transform(train['text'])
target_train = train['toxic']
features_test = count_tf_idf.transform(test['text'])
target_test = test['toxic']

model_tree = DecisionTreeClassifier(max_depth=10, class_weight='balanced', random_state=42) 
model_tree.fit(features_train, target_train)
predictions = model_tree.predict(features_test)
recall = recall_score(target_test, predictions)
precision = precision_score(target_test, predictions)
f1_sc = f1_score(target_test, predictions)
print(f'Полнота - {recall: .4f}, точность - {precision: .4f}, F1-мера - {f1_sc: .4f}')

Полнота -  0.4063, точность -  0.8723, F1-мера -  0.5544


Ага. Дерево заметно выигрывает у регрессии в точности - однако жутко просаживается по полноте, и итоговый результат нас, конечно, никак не устраивает. Попробуем поискать дальше:

In [24]:
features_train = train['text']
target_train = train['toxic']
features_test = test['text']
target_test = test['toxic']

tree_param = {'est__max_depth': [11, 13, 15, 17, 19, 21, 23]}

#gs_tree = GridSearchCV(tree_pipeline,
#                      param_grid=tree_param,
#                      scoring=make_scorer(f1_score),
#                      cv=5, verbose=0)
#gs_tree.fit(features_train, target_train)

#print(gs_tree.best_score_)
#print(gs_tree.best_params_)

`0.6081108797088997
{'est__max_depth': 23}`. Пробовать дальше не имеет смысла.

## <center>Заключение</center>

Также был попробован градиентный бустинг (LightGBM) - лучший результат F1-мера `0.738`, что также нас не устраивает. Без применения иных подходов (BERT и т.д.) лучший результат показывает логистическая регрессия с регуляризацией `C = 10`.